In [ ]:
import pandas as pd
import io
import getpass
import msoffcrypto
import numpy as np

In [ ]:
registry_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/pupillometry_sah/data/aSAH_DATA_2009_2023_24122023.xlsx'

In [ ]:
password = getpass.getpass()
decrypted_workbook = io.BytesIO()
with open(registry_data_path, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=password)
    office_file.decrypt(decrypted_workbook)

In [ ]:
registry_df = pd.read_excel(decrypted_workbook, sheet_name='DATA')

In [ ]:
for column in registry_df.columns:
    print(f'"{column}",')

In [ ]:
continuous_predictor_variables = [
    "Age",
    "GCS_admission",
    "Aneurysm_diameter"
]

In [ ]:
categorical_predictor_variables = [
    "Sex",
    "Smoker_0no_1yes_2ex",
    "Drinker",
"WFNS",
"HH",
    "CN_deficit_admission_YN",
"Sedation_on_admission_YN",
"Intubated_on_admission_YN",
"HTN",
"DM",
"Statin",
"ASS",
"Clopidogrel",
"OAC",
    "Fisher_Score",
"IVH",
"SDH",
"ICH",
"Multiple_Aneurysms_2unk",
"Nr_of_Aneurysms",
"Aneurysm_Artery_Code",
"Aneurysm_Side",
"Aneurysm_FusiformBlep_YN",
"Aneurysm_Distal_YN",
"Aneurysm_Mycotic_YN",
"Aneurysm_Blister_YN",
    "Coiling",
"Complete_Coiling",
"Clipping",
"Complete_Clipping",
    "Stenting",
    "EVD_YN",
    "CVS_YN",
"Nimotop"
]

In [ ]:
predictor_variables = categorical_predictor_variables + continuous_predictor_variables

In [ ]:
outcome_var = "DCI_YN"

In [ ]:
# fit xgb
import xgboost as xgb

model = xgb.XGBClassifier(enable_categorical=True)

In [ ]:
def safe_float_conversion(x):
    try:
        return float(x)
    except:
        return np.nan

In [ ]:
registry_df[categorical_predictor_variables] = registry_df[categorical_predictor_variables].astype('category')
registry_df['Age'] = registry_df['Age'].astype(float).apply(lambda x: np.abs(x))

for col in continuous_predictor_variables:
    registry_df[col] = registry_df[col].apply(safe_float_conversion)

In [ ]:
registry_df_with_outcome = registry_df[~registry_df[outcome_var].isna()]

In [ ]:
model.fit(registry_df_with_outcome[predictor_variables], registry_df_with_outcome[outcome_var])

In [ ]:
model.predict(registry_df_with_outcome[predictor_variables])

In [ ]:
model.predict_proba(registry_df_with_outcome[predictor_variables])[:,1]

In [ ]:
from static_prediction.evaluate_model import evaluate_model

evaluate_model(model, registry_df_with_outcome[predictor_variables], registry_df_with_outcome[outcome_var], outcome_var)